In [71]:
import opennre
import itertools
from utils import wikifier, coref_resolution, strip_punctuation
import spacy
import neuralcoref
import nltk
import transformers
from config import APPO, STOPWORDS, TOKENIZER
import re

relation_model = opennre.get_model('wiki80_bertentity_softmax')

2021-03-18 18:40:21,512 - root - INFO - Loading BERT pre-trained checkpoint.


In [2]:
#nlp = spacy.load("en_core_web_sm")
#doc = nlp(text)
#doc._.has_coref
#doc._.coref_clusters
#neuralcoref.add_to_pipe(nlp)

In [40]:
text = "Elon Musk is a business magnate, industrial designer, and engineer. Elon Musk is the founder, CEO, CTO, and chief designer of SpaceX. Elon Musk is also early investor, CEO, and product architect of Tesla , Inc. Elon Musk is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. Elon Musk briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen 's University. Elon Musk transferred to the University of Pennsylvania two years later, where Elon Musk received dual bachelor's degrees in economics and physics. Elon Musk moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. Elon Musk went on co-founding a web software company Zip2 with Elon Musk brother Kimbal Musk."

text = coref_resolution(text)
text

"Elon Musk is a business magnate, industrial designer, and engineer. Elon Musk is the founder, CEO, CTO, and chief designer of SpaceX. Elon Musk is also early investor, CEO, and product architect of Tesla , Inc. Elon Musk is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. Elon Musk briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen 's University. Elon Musk transferred to the University of Pennsylvania two years later, where Elon Musk received dual bachelor's degrees in economics and physics. Elon Musk moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. Elon Musk went on co-founding a web software company Zip2 wit

In [57]:
ner = transformers.pipeline('ner', grouped_entities=True)
entities = ner(text)

In [58]:
subjects = dict()
for entity in entities:
    if entity["word"] not in subjects:
        subjects[entity["word"]] = {
            "type": set([entity["entity_group"]])
        }
    else:
        subjects[entity["word"]]["type"].add(entity["entity_group"])

# Find matching indices
for key, _ in subjects.items():

    key_len = len(key)
    pos = [(m.start(), m.start() + key_len) for m in re.finditer(key, text)]
    if "pos" not in subjects:
        subjects[key]["pos"] = pos
    else:
        subjects[key]["pos"] += pos

In [69]:
triples = set()
#triples = dict()
relation_threshold = 0.9
text = strip_punctuation(text)

for subj in subjects.keys():
    for obj in subjects.keys():

        if subj == obj:
            continue

        for pos_subject in subjects[subj]["pos"]:
            for pos_object in subjects[obj]["pos"]:
                data = relation_model.infer({'text': text, 'h': {'pos': pos_subject}, 't': {'pos': pos_object}})

                if data[1] > relation_threshold:
                    triples.add((subj, data[0], obj))
                    #if triple in triples:
                    #    triples[triple].append(data[1])
                    #else:
                    #    triples[triple] = [data[1]]


In [72]:
triples

{('Elon Musk', 'part of', 'SpaceX'): [0.8604450821876526],
 ('Elon Musk', 'has part', 'SpaceX'): [0.934472382068634,
  0.9686380624771118,
  0.9686380624771118,
  0.9686380624771118,
  0.9686380624771118,
  0.9686380624771118,
  0.9686380624771118],
 ('Elon Musk', 'owned by', 'The Boring Company'): [0.9092851281166077,
  0.93291175365448,
  0.9336856603622437],
 ('Elon Musk', 'subsidiary', 'The Boring Company'): [0.8470849990844727,
  0.8470849990844727,
  0.8470849990844727,
  0.8470849990844727,
  0.8470849990844727,
  0.8470849990844727],
 ('Elon Musk', 'owned by', 'Neuralink'): [0.8015292286872864,
  0.744672954082489],
 ('Elon Musk', 'has part', 'Musk'): [0.9909204244613647,
  0.9210084080696106,
  0.9637876152992249,
  0.9592373371124268,
  0.7351576685905457,
  0.8954077959060669,
  0.7805033326148987,
  0.7093187570571899,
  0.7093187570571899,
  0.7093187570571899,
  0.7093187570571899,
  0.7093187570571899,
  0.7093187570571899],
 ('SpaceX', 'has part', 'Elon Musk'): [0.89464